## Домашняя работа №3

In [1]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras.layers import Dense, InputLayer
from tensorflow.keras.models import Sequential
import tensorflow.keras.metrics

In [2]:
print('TensorFlow version is ' + tf.__version__)

TensorFlow version is 2.5.0


In [3]:
boston_data = load_boston()
df_boston = pd.DataFrame(boston_data.data, columns=boston_data.feature_names)
df_boston['target'] = pd.Series(boston_data.target)
df_boston.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,target
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


In [4]:
X, y = load_boston(return_X_y=True)
print(y.shape)

(506,)


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [6]:
X = tf.constant( X_train , dtype=tf.float32 )
Y = tf.constant( y_train , dtype=tf.float32 ) 

In [7]:
num_features = X_train.shape[1]
num_classes = len(np.unique(y_train))

In [8]:
normalizer = preprocessing.Normalization(axis=-1)
normalizer.adapt(np.array(X))

In [9]:
first = np.array(X_train[:1])

with np.printoptions(precision=2, suppress=True):
  print('First example:', first)
  print()
  print('Normalized:', normalizer(first).numpy())

First example: [[  0.03   0.     2.18   0.     0.46   6.43  58.7    6.06   3.   222.
   18.7  394.12   5.21]]

Normalized: [[-0.41 -0.51 -1.29 -0.28 -0.85  0.15 -0.37  1.08 -0.75 -1.11  0.19  0.4
  -1.02]]


Построение первой модели

In [10]:
model = Sequential([
  normalizer,
  Dense(128, activation='tanh', input_shape=(num_features,)),
  Dense(128, activation='tanh'),
  Dense(1),
])


model.compile(
  optimizer='SGD',
  loss='mean_squared_error',
  metrics=['mae'],
)

model_fit = model.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2, verbose=0)

In [11]:
def history_last(model):
    last = list(model.history.history.values())[-1][-1]
    return last

In [12]:
first_note = [{"name": 'model', "loss": 'mse', 'metrics': 'mae', 'metric_value': history_last(model)}]

In [13]:
model_df = pd.DataFrame(first_note)

In [35]:
model_df

,name,loss,metrics,metric_value
0,model,mse,mae,2.251323
1,model1,logcosh,mae,2.578054
2,model2,mse,mae,2.217207
3,model3,msle,mae,4.075542
4,model4,msle,mae,3.881510
5,model5,mse,mape,11.108685
6,model6,mse,rmse,3.328029
7,model7,mae,mae,2.534641


In [14]:
model1 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='tanh'),
  Dense(1),
])


model1.compile(
  optimizer='SGD',
  loss='logcosh',
  metrics=['mae'],
)

model_fit = model1.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2, verbose=0)

In [15]:
model_df.loc[len(model_df)] = ['model1', 'logcosh', 'mae', history_last(model1)]

In [16]:
model2 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model2.compile(
  optimizer='SGD',
  loss='mean_squared_error',
  metrics=['mae'],
)

model_fit = model2.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2)

Epoch 1/50
19/19 [==============================] - 1s 15ms/step - loss: 287.2487 - mae: 13.8504 - val_loss: 111.6963 - val_mae: 8.0258
Epoch 2/50
19/19 [==============================] - 0s 3ms/step - loss: 307.6246 - mae: 14.1640 - val_loss: 63.7324 - val_mae: 5.6007
Epoch 3/50
19/19 [==============================] - 0s 3ms/step - loss: 198.4789 - mae: 11.1290 - val_loss: 112.6451 - val_mae: 7.6729
Epoch 4/50
19/19 [==============================] - 0s 3ms/step - loss: 91.5591 - mae: 6.8532 - val_loss: 50.9297 - val_mae: 5.2629
Epoch 5/50
19/19 [==============================] - 0s 3ms/step - loss: 42.4825 - mae: 4.6003 - val_loss: 45.6961 - val_mae: 4.5926
Epoch 6/50
19/19 [==============================] - 0s 3ms/step - loss: 27.6305 - mae: 3.8778 - val_loss: 78.6783 - val_mae: 6.6188
Epoch 7/50
19/19 [==============================] - 0s 4ms/step - loss: 42.2358 - mae: 4.4659 - val_loss: 51.0925 - val_mae: 4.9630
Epoch 8/50
19/19 [==============================] - 0s 3ms/step - l

In [17]:
model_df.loc[len(model_df)] = ['model2', 'mse', 'mae', history_last(model2)]

In [18]:
model3 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model3.compile(
  optimizer='SGD',
  loss='mean_squared_logarithmic_error',
  metrics=['mae'],
)

model_fit = model3.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2, verbose=0)

In [19]:
model_df.loc[len(model_df)] = ['model3', 'msle', 'mae', history_last(model3)]

In [20]:
model4 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model4.compile(
  optimizer='SGD',
  loss='mean_squared_logarithmic_error',
  metrics=['mae'],
)

model_fit = model4.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2, verbose=0)

In [21]:
model_df.loc[len(model_df)] = ['model4', 'msle', 'mae', history_last(model4)]

In [23]:
model5 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model5.compile(
  optimizer='SGD',
  loss='mean_squared_error',
  metrics=[tf.keras.metrics.MeanAbsolutePercentageError()],
)

model_fit = model5.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2)

Epoch 1/50
19/19 [==============================] - 0s 9ms/step - loss: 472.5196 - mean_absolute_percentage_error: 78.7242 - val_loss: 361.5710 - val_mean_absolute_percentage_error: 71.7501
Epoch 2/50
19/19 [==============================] - 0s 4ms/step - loss: 299.9220 - mean_absolute_percentage_error: 58.9338 - val_loss: 172.2305 - val_mean_absolute_percentage_error: 42.3813
Epoch 3/50
19/19 [==============================] - 0s 3ms/step - loss: 191.4231 - mean_absolute_percentage_error: 43.5257 - val_loss: 180.8069 - val_mean_absolute_percentage_error: 43.0956
Epoch 4/50
19/19 [==============================] - 0s 3ms/step - loss: 101.3282 - mean_absolute_percentage_error: 27.8217 - val_loss: 116.0933 - val_mean_absolute_percentage_error: 31.2778
Epoch 5/50
19/19 [==============================] - 0s 3ms/step - loss: 52.7104 - mean_absolute_percentage_error: 22.0087 - val_loss: 38.5469 - val_mean_absolute_percentage_error: 20.3824
Epoch 6/50
19/19 [==============================] - 

In [24]:
model_df.loc[len(model_df)] = ['model5', 'mse', 'mape', history_last(model5)]

In [28]:
model6 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model6.compile(
  optimizer='SGD',
  loss='mean_squared_error',
  metrics=[tf.keras.metrics.RootMeanSquaredError()],
)

model_fit = model6.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2)

Epoch 1/50
19/19 [==============================] - 0s 8ms/step - loss: 360.2763 - root_mean_squared_error: 18.9809 - val_loss: 381.5254 - val_root_mean_squared_error: 19.5327
Epoch 2/50
19/19 [==============================] - 0s 3ms/step - loss: 240.3837 - root_mean_squared_error: 15.5043 - val_loss: 79.2591 - val_root_mean_squared_error: 8.9028
Epoch 3/50
19/19 [==============================] - 0s 3ms/step - loss: 228.9868 - root_mean_squared_error: 15.1323 - val_loss: 129.9556 - val_root_mean_squared_error: 11.3998
Epoch 4/50
19/19 [==============================] - 0s 3ms/step - loss: 77.4242 - root_mean_squared_error: 8.7991 - val_loss: 93.4235 - val_root_mean_squared_error: 9.6656
Epoch 5/50
19/19 [==============================] - 0s 3ms/step - loss: 81.3543 - root_mean_squared_error: 9.0197 - val_loss: 70.0985 - val_root_mean_squared_error: 8.3725
Epoch 6/50
19/19 [==============================] - 0s 3ms/step - loss: 75.6203 - root_mean_squared_error: 8.6960 - val_loss: 178.

In [29]:
model_df.loc[len(model_df)] = ['model6', 'mse', 'rmse', history_last(model6)]

In [33]:
model7 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model7.compile(
  optimizer='SGD',
  loss='mae',
  metrics=['mae'],
)

model_fit = model7.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2)

Epoch 1/50
19/19 [==============================] - 0s 9ms/step - loss: 22.4221 - mae: 22.4221 - val_loss: 21.1780 - val_mae: 21.1780
Epoch 2/50
19/19 [==============================] - 0s 3ms/step - loss: 20.1096 - mae: 20.1096 - val_loss: 18.0042 - val_mae: 18.0042
Epoch 3/50
19/19 [==============================] - 0s 3ms/step - loss: 15.3041 - mae: 15.3041 - val_loss: 10.6753 - val_mae: 10.6753
Epoch 4/50
19/19 [==============================] - 0s 3ms/step - loss: 8.6538 - mae: 8.6538 - val_loss: 5.4600 - val_mae: 5.4600
Epoch 5/50
19/19 [==============================] - 0s 3ms/step - loss: 5.2688 - mae: 5.2688 - val_loss: 3.8701 - val_mae: 3.8701
Epoch 6/50
19/19 [==============================] - 0s 3ms/step - loss: 3.6933 - mae: 3.6933 - val_loss: 3.4258 - val_mae: 3.4258
Epoch 7/50
19/19 [==============================] - 0s 3ms/step - loss: 3.1891 - mae: 3.1891 - val_loss: 3.1610 - val_mae: 3.1610
Epoch 8/50
19/19 [==============================] - 0s 3ms/step - loss: 2.9803

In [34]:
model_df.loc[len(model_df)] = ['model7', 'mae', 'mae', history_last(model7)]

In [36]:
model8 = Sequential([
  normalizer,
  Dense(128, activation='relu', input_shape=(num_features,)),
  Dense(128, activation='relu'),
  Dense(1),
])


model8.compile(
  optimizer='SGD',
  loss='mae',
  metrics=[tf.keras.metrics.MeanAbsolutePercentageError()],
)

model_fit = model8.fit(X, Y, epochs=50, batch_size=15, validation_split=0.2)

Epoch 1/50
19/19 [==============================] - 1s 20ms/step - loss: 22.0690 - mean_absolute_percentage_error: 94.2710 - val_loss: 20.6695 - val_mean_absolute_percentage_error: 89.0458
Epoch 2/50
19/19 [==============================] - 0s 3ms/step - loss: 19.1751 - mean_absolute_percentage_error: 78.7179 - val_loss: 16.7898 - val_mean_absolute_percentage_error: 70.7766
Epoch 3/50
19/19 [==============================] - 0s 3ms/step - loss: 13.4790 - mean_absolute_percentage_error: 55.6470 - val_loss: 8.1514 - val_mean_absolute_percentage_error: 35.4589
Epoch 4/50
19/19 [==============================] - 0s 4ms/step - loss: 7.2229 - mean_absolute_percentage_error: 34.3030 - val_loss: 5.2353 - val_mean_absolute_percentage_error: 24.6907
Epoch 5/50
19/19 [==============================] - 0s 4ms/step - loss: 4.8656 - mean_absolute_percentage_error: 23.9482 - val_loss: 4.3010 - val_mean_absolute_percentage_error: 20.2996
Epoch 6/50
19/19 [==============================] - 0s 3ms/step 

In [37]:
model_df.loc[len(model_df)] = ['model8', 'mae', 'mape', history_last(model8)]

In [38]:
model_df

,name,loss,metrics,metric_value
0,model,mse,mae,2.251323
1,model1,logcosh,mae,2.578054
2,model2,mse,mae,2.217207
3,model3,msle,mae,4.075542
4,model4,msle,mae,3.881510
5,model5,mse,mape,11.108685
6,model6,mse,rmse,3.328029
7,model7,mae,mae,2.534641
8,model8,mae,mape,12.637752


Из документации взял команды нормализации, так же очень много посидел в разделах Loss и metrics, понял что многие метрики не подходят для задачи регрессии (напрмиер, такие как, accuracy). По таблице лучше всего показали loss: mse, metrics: mae.